In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy
import pandas as pd
import os
import time
import itertools
import lightkurve as lk
import pickle

## EB Catalogs: Messing Around
In this notebook, I am messing around with some catalogs of eclipsing binaries (EBs) that I located. Here are the catalogs I've found (not all of which are used in this notebook). Those for which I have the data in files are in the "catalogs" directory of the Github repository:
* 748 EBs that are mainly detached binaries; has TIC IDs available in the data (Justesen & Albrecht 2021). Not filtered by magnitude, but it has Tmag as a column so I can sort and filter in Python. Filename: ‘justesen_albrecht_table2_748ebs.txt’
* 2156 EBs from TESS; has TIC IDs available ([see here](tessebs.villanova.edu)). Filename: ‘tess_ebs_villanova_tmag_10-15.csv’
* 1997 EBs from Kepler between Kepler magnitudes 10 and 15; has KIC IDs but no TIC IDs. Since they’re from Kepler, they’re focused on a particular (small) area of the sky ([see here](keplerebs.villanova.edu); based off of Kirk et al. 2016). Filename: ‘keplerebs_villanova_kmag_10-15.csv’
* 1459 EBs from OGLE catalog – these are also biased towards well-detached systems. All are in a central 2.4 sq. deg. area of the Small Magellanic Cloud and have I-band mag < 20, with periods between 0.3 and 250 days. Most appear to have I-band mags > 15 (see Fig. 3 of Udalski et al. 1998) – this makes sense since they’re technically extragalactic. Also, 68 stars were listed twice due to appearing in multiple sectors, which is why the tables are 1527 lines long – see Table 3 for which identifiers refer to the same star (Udalski et al. 1998)
* 20,000 EBs from SIMBAD; these can be unreliable and their listed names are not organized (no TIC or KIC IDs). No magnitude filtering on these although I could do a new query to get the mags concentrated between 10 and 15. Filename: ‘simbad_eclipsing_binaries_20000_stars.txt’ \[Note: format on this file is a little funky; it’s how SIMBAD outputs it. See my eclipsing binary test Jupyter notebook on VM for how I sort of unpacked it one time.\]
* 269 EBs from the DEBCat catalog (“detached eclipsing binary catalog”), available [here](https://www.astro.keele.ac.uk/jkt/debcat/) and saved as “debcat_ebs.txt”. These are all detached EBs (no contact binaries, for example). No magnitude filtering done on these but they do have V-band mag info included. No TIC ID though, and no RA and dec so I’d have to get creative with how to get TIC IDs for these. Probably it would be something like querying SIMBAD with the names listed and getting from it either the TIC ID or the RA and dec, which I could then plug into tesspoint/tess2px to get the TIC ID (or could use the [Sesame name resolver](http://cds.u-strasbg.fr/cgi-bin/Sesame) to go right from names to TIC ID?)
* The All-Sky Automated Survey (ASAS) Catalogue of Variable Stars can be queried [here](http://www.astrouw.edu.pl/asas/?page=acvs) and has options for what types of eclipsing variables to query – note that they are all EBs and that EC = eclipsing contact, ESD = eclipsing semidetached, ED = eclipsing detached
* The General Catalog of Variable Stars (GCVS) has a lot of eclipsing binaries; see http://www.sai.msu.su/gcvs/gcvs/
* International Variable Star Index (VSX), established in 2007 by the AAVSO

In [2]:
# Mount the GCP filesystem onto this VM
data_dir = "/home/parsellsx/tesslcs/"
os.system(f"gcsfuse --implicit-dirs tess-goddard-lcs {data_dir}")

256

Let's first try messing around with the Justesen & Albrecht (2021) catalog: can open it up with either Pandas, NumPy, or base Python. I'll try Pandas.

In [3]:
# Note that 'skipinitialspace=True' makes it so that the space at the start of the TIC ID in the file doesn't 
# make pandas interpret it as a NaN. Also, index_col=False makes it not interpret the TIC ID column as indices
# and instead treats it as actual data (which it is)
justesen = pd.read_table('justesen_albrecht_table2_748ebs.txt',sep=' ',header=None,
                     names=['TIC ID','Period','t1','t2','ecosw','d1','d2','Tmag'],index_col=False,skiprows=21,
                    skipinitialspace=True)

In [ ]:
justesen

In [ ]:
justesen['Tmag']

In [ ]:
justesen['Tmag'][3:10]

In [ ]:
good_mags = np.where(np.logical_and(justesen['Tmag'] > 10, justesen['Tmag'] < 15))[0]

In [ ]:
print(good_mags.size)

OK, so we have 440 objects from this catalog that satisfy our magnitude constraints (10 < m < 15). Let's see how we do with some of the other catalogs. Check out the Villanova TESS EB catalog.

In [ ]:
tessebs = pd.read_csv('tess_ebs_villanova_tmag_10-15.csv',header=None,names=['TIC ID','Signal ID','BJD0','BJD0_uncert','Period','Period_uncert','Morph','Morph_dist','RA','Dec','Tmag','GLon','GLat','Teff','Log g','Abundance'],index_col=False,skiprows=1)

In [ ]:
tessebs

All these EBs are in the TIC and they're all in the magnitude range I'm interested in. So between this catalog and the Justesen & Albrecht one above, we have 2596 EBs identified (although there could be overlap - could be as few as 2156). The question is, how many did TESS actually observe/how many do we have light curves for? 

To figure this out, we should use the lookup tables that are in our Google Cloud buckets. Want to do two things: 1) Get the actual filenames associated with each of our TIC IDs, and 2) Identify any duplicates between catalogs.

I've mounted the GCP filesystem on my VM's filesystem, so I should be able to access the lookup files pretty easily from here.

In [ ]:
# Maybe in order to get rid of duplicates what I should do is just get all the filenames acting as though there's 
# no duplicates anywhere, then once I have everything, run some function that just identifies duplicates and gets
# rid of them. There's probably one pre-built into NumPy or something like that

# Lookup tables: filename, RA, dec, TIC ID, sector, camera, CCD, magnitude
# How to deal with the fact that there are 26 different lookup tables and I don't really know what sector each 
# listed EB is in? I could conceivably get the boundaries of each sector in ecliptic coordinates, then convert 
# each EB's RA/dec to that coordinate system and determine what sector it's in that way, then check the correct
# lookup table file to get its filename. Another option would be just looking through every sector, every time, but
# that would probably take a really long time. Let's do a test just to see how long it takes for one ID that's in 
# sector 13 (meaning it would probably be about the average lookup time if there's an equal number of EBs in 
# sectors 1-13 as in 14-26).
lookup14 = pd.read_csv('~/tesslcs/sector14lookup.csv',header=None,names=['filename','RA','dec','TIC ID',
                                                    'sector','camera','CCD','mag'],index_col=False,skiprows=1)
print(lookup14['TIC ID'][0])

In [ ]:
test_lookup = lookup14['TIC ID'][0]

In [ ]:
%%script false
# Awesome - now it works like it should. So let's take this TIC ID and "search" through all the lookup files until
# we find it, and get the corresponding filename
# I see two ways to do this: either read in all the files in advance and keep them in memory (which seems like it
# would maybe be memory-intensive enough to crash the VM, but what the hell, I guess I can always add more memory
# if I need it) or I can only open and keep in memory the one I'm currently looking through. That would take way 
# longer, I think, to do this for a lot of objects like I want to do. Let's try it the first way:

# We can save memory by only reading in the columns we need - TIC ID and filename
lookuplist = [] # List to hold all the dataframes
names = ['filename','RA','dec','TIC ID','sector','camera','CCD','mag']
for i in range(1,27):
    print(i)
    lookup = pd.read_csv('~/tesslcs/sector' + str(i) + 'lookup.csv',header=None,names=names,index_col=False,
                        skiprows=1,usecols=['filename','TIC ID'])
    lookuplist.append(lookup)

When I tried the above, it made it to sector 10 and then killed the kernel because it was too much data. So it's not going to work like that unless I upgrade the memory on the VM. That's definitely possible but it'd be nice to get it to work without doing that.

So I guess the next thing to try would be to actually just read through one lookup table file at a time until we find the TIC ID we're looking for, and then do that same thing for every single TIC ID that we want to get a filename for. 

If this fails, there are two options that I see right away: 1) give my VM more memory and try again the first way, or 2) actually write some script (which would probably be a little complex) to take the coordinates of a star and transform that into its TESS sector (then I would only need to check one lookup table for each star).

In [ ]:
print(time.ctime())
test_filename = '' # This will contain the filename of the LC of the TIC ID stored in test_lookup
names = ['filename','RA','dec','TIC ID','sector','camera','CCD','mag']
test_lookup = str(test_lookup) + '.pkl'
for i in range(1,27):
    # Now test_lookup is a specific TIC ID that I want to find, and I know it's in sector 14 but for the purposes
    # of this test I'm going to pretend I don't know that. Can we find it?
    lookuptable = pd.read_csv('~/tesslcs/sector' + str(i) + 'lookup.csv',header=None,names=names,index_col=False,
                             skiprows=1,usecols=['TIC ID'],dtype=str)
    print(i)
    # Two options I can think of: either 1) read in only the TIC ID column and just search that (might be easier 
    # actually because we can use NumPy-style search tools) and then if the TIC ID is in there, then read in the 
    # filename column separately, OR 2) read in both columns for every lookup file. My gut is saying 1) will be
    # faster
    lookuptable['Indexes'] = lookuptable['TIC ID'].str.find(test_lookup) # Make new column - is it ever not -1?
    id_inds = np.where(lookuptable['Indexes'] != -1)[0]
    if id_inds.size > 0: # I.e., if it finds the TIC ID in the current file
        # If the TIC ID is in the current lookup table, then we want to record the filename somewhere
        id_index = id_inds[0] # Get line number of this TIC ID
        if id_inds.size > 1:
            print('More than one LC of this TIC ID in this sector.')
        test_filename = pd.read_csv('~/tesslcs/sector' + str(i) + 'lookup.csv',header=None,names=names,
                           index_col=False,skiprows=1,usecols=['filename'],dtype=str)['filename'][id_index]
        break # We found that TIC ID, so no need to keep looking afterwards

In [ ]:
print(time.ctime())

In [ ]:
test_filename

While testing the above, I noticed that if you just search for the actual TIC ID, you might find a different TIC ID in one of the lookup tables that is one digit longer but contains the ID you searched for. That's obviously not desirable, so I recommend searching for the string "27693449.pkl" if you're looking for the TIC ID 27693449 (don't just search for the string "27693449").

I just now (July 6, 12:50 am) updated my GCP VM to have 4 CPUs and 16 GB RAM, so I'm going to try my original idea again, where I just load in all the lookup files into one big array and then search that for the TIC ID. Let's see if it can handle it this time (before it had only 4 GB). Also I just realized, I don't even need to be reading in the TIC ID column since the TIC ID is included in the filename.

In [ ]:
lookuplist = [] # List to hold all the dataframes
names = ['filename','RA','dec','TIC ID','sector','camera','CCD','mag']
for i in range(1,27):
    print(i)
    lookup = pd.read_csv('~/tesslcs/sector' + str(i) + 'lookup.csv',header=None,names=names,index_col=False,
                        skiprows=1,usecols=['filename','TIC ID'],dtype=str)
    lookuplist.append(lookup)

In [ ]:
# Nice - that worked! Only took maybe 3 min or so, which is reasonable
# Want to find test_lookup in that big list of dataframes
test_string = str(test_lookup) + '.pkl'
test_filename = '' # This will store the full filename/filepath to get to the LC for the current TIC ID
for i in range(0,26):
    print(i)
    for j in lookuplist[i]['filename']:
        if test_string in j:
            test_filename = j

In [ ]:
test_filename

That was unexpected - I put in a TIC ID that I know should be in sector 14, but it returned a filepath to a LC in sector 16. I'm thinking what's happening is that this star is listed in both sector 14 and 16 - let's see if I'm right by rerunning the above but modifying it so it prints out when it finds the star I'm looking for.

In [ ]:
test_string = str(test_lookup) + '.pkl'
test_filename = '' # This will store the full filename/filepath to get to the LC for the current TIC ID
for i in range(0,26):
    print(i)
    for j in lookuplist[i]['filename']:
        if test_string in j:
            test_filename = j
            print(test_filename)

OK, looks like I was right about that. So how do we handle the fact that the same star shows up in multiple sectors? I first want to check and see if the lightcurves at each of these 3 filepaths are the same or different - I'm guessing they're different, but if they're the same, then obviously that solves the problem right there because I can just use the first one.

In [ ]:
import lightkurve as lk
import pickle
fp = open('../tesslcs/' + test_filename,'rb')
data = pickle.load(fp)
fp.close()
type(data)

In [ ]:
filename_list = ['tesslcs_sector_14_104/2_min_cadence_targets/tesslc_27693449.pkl',
                 'tesslcs_sector_15_104/tesslcs_tmag_9_10/tesslc_27693449.pkl',
                 'tesslcs_sector_16_104/tesslcs_tmag_9_10/tesslc_27693449.pkl']
for x in filename_list:
    fp = open('../tesslcs/' + x,'rb')
    data = pickle.load(fp)
    fp.close()
    print(data[9][:10]) # PCA flux, first 10 measurements - let's see if they're the same for all 3 files

OK, so we see here that they are not, in fact, the same light curves. Interesting. So I guess what I can do is instead of going through all 26 sectors and picking out the first light curve that matches a given TIC ID, I can pull out _all_ the light curves that match a given TIC ID, and include them all in my training data.

What's next? My goal here is to get the filenames for every object in my EB catalogs. So let's do that now - let's start with the two catalogs that I actually have TIC IDs for already (i.e., Justesen & Albrecht and the tessebs Villanova catalog). My goal will be to make a single list or array which just contains a bunch of filenames of light curves that are EBs, and then after filtering out duplicates, just save that list to a text file.

In [ ]:
def TICID_to_filepath(TICID,lookuplist=lookuplist):
    # Takes in a TIC ID, searches the GCP bucket for any corresponding filepaths (could be multiple) and returns
    # a list (which can be empty) containing all such filepaths.
    id_filelist = [] 
    search_str = '_' + str(TICID) + '.pkl' # This is the string we'll actually look for in lookuplist, our df list
    # We include the underscore in search_str to avoid getting other TIC IDs that contain the TIC ID we're looking
    # for but have another digit in front of it
    for i in range(0,26): # Loop through every dataframe in lookuplist (i.e., every lookup table file)
        for j in lookuplist[i]['filename']:
            if search_str in j:
                id_filelist.append(j)
    return id_filelist

In [ ]:
print(time.ctime())
TICID_to_filepath(test_lookup)
print(time.ctime())

In [ ]:
filelist = [] # This will hold all the filepaths to the light curves corresponding to our catalog TIC IDs
# Our two data "sets" that we want to iterate through right now are justesen['TIC ID'][good_mags] and 
# tessebs['TIC ID']. We'll count the number of stars that are found in our GCP bucket LCs and compare that to the
# number of stars in the catalog as well as the final size of filelist (which will depend on how many LCs exist
# for a single star on average)
for i in justesen['TIC ID'][good_mags]:
    filelist.extend(TICID_to_filepath(i)) # Get the filepath(s) for this TIC ID, then append to our list of paths
for i in tessebs['TIC ID']:
    filelist.extend(TICID_to_filepath(i))
# Now write filelist to a text file so we have it for later. Remember these are all EBs. Remember also that we
# haven't filtered out duplicates yet
with open('eb_filepath_list_justesen_tessebs_with_duplicates.txt', 'w') as f:
    for item in filelist:
        f.write("%s\n" % item)
print(len(filelist))
print("We put in 2596 TIC IDs")

In [ ]:
# Want to figure out how to use itertools.chain() instead of list.extend() above because I hear it's faster
itertools.chain(x,y)